In [1]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from TimingCalib.MPMT import MPMT
from TimingCalib.PMT import PMT
from TimingCalib.LED import LED

from scipy.spatial.transform import Rotation as R

In [4]:
my_mpmt = MPMT(None, 'M2', {}, {})

origins = []
vectors = []
n_mp = 20
meshes = []
radius = my_mpmt.pmts[0].prop_design['size']/2.

for i_pmt, pmt in enumerate(my_mpmt.pmts):
    location, direction_x, direction_z = pmt.get_orientation('true')

    lv = [direction_z[i]*100 for i in range(len(direction_z))]
    origins.append(location)
    vectors.append(lv)

    mesh = []
    if i_pmt > -1:
        # draw the surface of the pmt with n_mp points
        # find a direction perpendicular to normal p = (px, 0., 1.)
        px = -direction_z[2]/(direction_z[0]+1.E-3)
        # change magnitude to PMT radius
        scale = radius/np.sqrt(1. + px**2)
        perp = (px*scale,0.,scale)

        rot = R.from_rotvec(2.*np.pi/n_mp * np.array(direction_z))
        for i in range(n_mp):
            mesh.append(np.add(location,perp))
            perp = rot.apply(perp)
        meshes.append(np.array(mesh,dtype=np.float32))


In [7]:
# plot the normal vectors

plt_vectors = k3d.vectors(origins=origins, vectors=vectors, head_size=50.)

plot = k3d.plot()
plot += plt_vectors
plot.display()

Output()

In [8]:
# show the PMT faces with numbering

indices = []
for i in range(n_mp):
    indices.append([i,(i+1)%n_mp])


for i_mesh,mesh in enumerate(meshes):
    plt_mesh = k3d.lines(mesh, indices, indices_type='segment')
    plot += plt_mesh
    plt_text = k3d.text(str(i_mesh), position=origins[i_mesh], reference_point='cc', size=1., label_box=False)
    plot += plt_text

plot.display()

/gpfs/fs1/home/k/karlen/karlen/wcte/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [15]:
# Draw the components of a single mPMT using k3d

plot = k3d.plot()

placement = {'loc':[100.,100.,0.], 'loc_sig':[0.001,0.001,0.001], 'rot_axes':'Z', 'rot_angles':[0.], 'rot_angles_sig':[1E-6]}

# make a standard domed mPMT
my_mpmt = MPMT(None, 'M2', placement , placement)

# pmt coordinates
color_z = 0x28b463
color_x = 0x2874a6
origins = []
z_vecs = []
x_vecs = []
vec_length = 10. # length of x,z vectors to show for each pmt

# pmt faces:
color_pmt = 0xdc7633
n_point_pmt = 20
indices_pmt = []
for i in range(n_point_pmt):
    indices_pmt.append([i,(i+1)%n_point_pmt])

# mpmt
color_mpmt = 0xabb2b9
n_point_mpmt = 100
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

# draw the extent of an mpmt
circle_points = np.array(my_mpmt.get_circle_points(n_point_mpmt,'design'),dtype=np.float32)
plt_circle = k3d.lines(circle_points, indices_mpmt, indices_type='segment', color=color_mpmt)
plot += plt_circle

for i_pmt, pmt in enumerate(my_mpmt.pmts):
    location, direction_x, direction_z = pmt.get_orientation('design')

    # lists to show PMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)

    # k3d expects ndarray of float32 for segments
    circle_points = np.array(pmt.get_circle_points(n_point_pmt,'design'),dtype=np.float32)
    plt_circle = k3d.lines(circle_points, indices_pmt, indices_type='segment', color=color_pmt)
    plot += plt_circle
    plt_text = k3d.text(str(i_pmt), position=location, reference_point='cc', size=1., label_box=False, color=color_pmt)
    plot += plt_text

# plot all pmt coordinate axes
plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z)
plot += plt_z_vecs
plt_x_vecs = k3d.vectors(origins=origins, vectors=x_vecs, color=color_x)
plot += plt_x_vecs

plot.display()

Output()